In [ ]:
import joblib
import contextlib
from tqdm import tqdm
from joblib import Parallel, delayed

import numpy as np
import itertools
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

from scipy.integrate import quad
from scipy.special import gamma
import math

import time

In [ ]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""

    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback

    try:
        yield tqdm_object

    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [ ]:
TARG_LST = [
    'double_well',
    'gaussian',
    'gaussian_mixture'
]

ALGO_LST = [
    'LMC',
    'LMCO',
    'aHOLLA',
    'mTULA',
    'HOLA',
    'aHOLA'
]

class LangevinSampler:
    def __init__(self, targ, algo, step=0.001, beta=1, a=None, d=None):
        assert targ in TARG_LST
        assert algo in ALGO_LST
        
        self.targ = targ
        self.algo = algo
        self.step = step
        self.beta = beta
        self.a = a  # for mixed gaussian
        self.d = d  # dimension of the parameter
            
    
    def _gradient(self, theta):
        if self.targ == 'double_well':
            return (np.dot(theta, theta) - 1) * theta
        elif self.targ == 'gaussian':
            return theta
        elif self.targ == 'gaussian_mixture':
            return theta - self.a + 2 * self.a / (1 + np.exp(2 * np.dot(theta, self.a)))
                
    def _hessian(self,theta):
        if self.targ == 'double_well':
            return (np.dot(theta, theta) - 1) * np.eye(self.d) + 2 * np.outer(theta, theta)
        elif self.targ == 'gaussian':
            return np.eye(self.d)
        elif self.targ == 'gaussian_mixture':
            return np.eye(self.d) - 4 * np.outer(self.a, self.a) * np.exp(2 * np.dot(theta, self.a)) / (1 + np.exp(2 * np.dot(theta, self.a)))**2

    def _hessiangradproduct(self,theta):
        grad = self._gradient(theta)
        if self.targ == 'double_well':
            return (np.dot(theta, theta) - 1) * grad + 2 * theta * np.dot(theta, grad)
        elif self.targ == 'gaussian':
            return grad
        elif self.targ == 'gaussian_mixture':
            return grad - 4 * self.a * np.dot(self.a, grad) * np.exp(2 * np.dot(theta, self.a)) / (1 + np.exp(2 * np.dot(theta, self.a)))**2
    
    def _vectorlaplacian(self, theta):
        if self.targ == 'double_well':
            return 2 * (self.d + 2) * theta
        elif self.targ == 'gaussian':
            return np.zeros(self.d)
        elif self.targ == 'gaussian_mixture':
            return 8 * self.a * np.dot(self.a, self.a) * (np.exp(
                4 * np.dot(theta, self.a)) - np.exp(2 * np.dot(theta, self.a))) / (1 + np.exp(2 * np.dot(theta, self.a)))**3
            
    def _gradient_term(self, theta):
        grad = self._gradient(theta)
        if self.algo in ['LMC', 'LMCO', 'aHOLLA']:
            return grad
        elif self.algo == 'mTULA':
            return grad / np.sqrt(1 + self.step * (np.dot(theta, theta) ** 2))
        elif self.algo == 'HOLA':
            return grad / ((1 + (self.step ** (3/2)) * (np.linalg.norm(grad) ** (3/2))) ** (2/3))
        elif self.algo == 'aHOLA':
            return grad / ((1 + (self.step ** (3/2)) * (np.dot(theta, theta) ** 3)) ** (1/3))

    def _hessian_term(self, theta):
        grad = self._gradient(theta)
        hessian = self._hessian(theta)
        hvp = self._hessiangradproduct(theta)
        if self.algo in ['LMC', 'mTULA']:
            return 0
        elif self.algo in ['LMCO', 'aHOLLA']:
            return hvp
        elif self.algo == 'HOLA':
            return hvp / (1 + self.step * (np.linalg.norm(theta)) * (np.linalg.norm(hessian)) * (np.linalg.norm(grad)))
        elif self.algo == 'aHOLA':
            return hvp / ((1 + (self.step**(3/2))*(np.dot(theta, theta)**3))**(2/3))

    def _vectorlaplacian_term(self,theta):
        vectorlap = self._vectorlaplacian(theta)
        if self.algo in ['LMC', 'LMCO', 'mTULA']:
            return 0
        elif self.algo == 'aHOLLA':
            return vectorlap / self.beta
        elif self.algo == 'HOLA':
            return vectorlap / ((1 + (self.step**(1/2)) * (np.linalg.norm(theta)) * (np.linalg.norm(vectorlap))) * self.beta)
        elif self.algo == 'aHOLA':
            return vectorlap / ((1 + (self.step**(3/2))*(np.dot(theta, theta)**3))**(1/3) * self.beta)

    def _hessiangauproducta(self,theta,gau_a):
        if self.targ == 'double_well':
            return self.step * ((np.dot(theta, theta) - 1) * gau_a + 2 * theta * np.dot(theta, gau_a)) / 2
        elif self.targ == 'gaussian':
            return self.step * gau_a / 2
        elif self.targ == 'gaussian_mixture':
            return self.step * (gau_a - 
                                      4 * self.a * np.dot(self.a, gau_a) * np.exp(2 * np.dot(theta, self.a)) / (1 + np.exp(2 * np.dot(theta, self.a)))**2) / 2

    def _hessiangauproductb(self,theta,gau_b):
        if self.targ == 'double_well':
            return (np.sqrt(3) / 6) * self.step * ((np.dot(theta, theta) - 1) * gau_b + 2 * theta * np.dot(theta, gau_b))
        elif self.targ == 'gaussian':
            return (np.sqrt(3) / 6) * self.step * gau_b
        elif self.targ == 'gaussian_mixture':
            return (np.sqrt(3) / 6) * self.step * (gau_b - 
                                      4 * self.a * np.dot(self.a, gau_b) * np.exp(2 * np.dot(theta, self.a)) / (1 + np.exp(2 * np.dot(theta, self.a)))**2)
    def _diffusion_term(self,theta,gau_a,gau_b):
        hessian = self._hessian(theta)
        taming_1 = (1 + self.step * (np.linalg.norm(hessian)))
        taming_2 = ((1 + (self.step**(3/2))*(np.dot(theta, theta)**3))**(1/3))
        if self.algo in ['LMC', 'LMCO', 'mTULA']:
            return np.random.standard_normal(self.d)
        elif self.algo == 'aHOLLA':
            return gau_a - self._hessiangauproducta(theta,gau_a) + self._hessiangauproductb(theta,gau_b)
        elif self.algo == 'HOLA':
            return gau_a - self._hessiangauproducta(theta,gau_a) / taming_1 + self._hessiangauproductb(theta,gau_b) / taming_1
        elif self.algo == 'aHOLA':
            return gau_a - self._hessiangauproducta(theta,gau_a) / taming_2 + self._hessiangauproductb(theta,gau_b) / taming_2

    
    def sample(self, theta0, n_iter=10**5, n_burnin=10**4, return_arr=True, runtime=200):
        if runtime is not None:
            n_iter = int(runtime/self.step)
            n_burnin = n_iter
            
        theta = np.ravel(np.array(theta0).reshape(-1))
        
        if return_arr:
            theta_arr = np.zeros((n_iter + n_burnin, self.d))

        for n in np.arange(n_iter + n_burnin):
            gau_a = np.random.standard_normal(self.d)
            gau_b = np.random.standard_normal(self.d)
            theta = theta + (- self.step * self._gradient_term(theta) + 
                            (self.step ** 2 / 2) * self._hessian_term(theta) - 
                            (self.step ** 2 / 2) * self._vectorlaplacian_term(theta) + 
                            np.sqrt(2 * self.step / self.beta) * self._diffusion_term(theta,gau_a,gau_b))
            
            if return_arr:
                theta_arr[n] = theta

        return theta if (not return_arr) else theta_arr

In [ ]:
n_chains = 500
def draw_samples_parallel(sampler, theta0, runtime=200, n_chains=n_chains, n_jobs=-1):
    d = len(np.ravel(np.array(theta0).reshape(-1)))
    sampler.d = d
    
    def _run_single_markov_chain():
        # Extract only the first dimension (first column)
        return pd.DataFrame(
            sampler.sample(theta0, runtime=runtime)[:, 0],  # Change here to take only the first dimension
            columns=['component_1']
        )

    with tqdm(total=n_chains, desc=f"Running {sampler.algo} (d={d}, step={sampler.step})") as pbar:
        with tqdm_joblib(pbar):
            samples_df_1st = Parallel(n_jobs=n_jobs)(
                delayed(_run_single_markov_chain)() for _ in range(n_chains)
            )

    return pd.concat(samples_df_1st, ignore_index=True)

In [ ]:
# Define parameters
d_values = [100]#[2,10,100]
gaussian_steps = [0.1, 0.05, 0.025] #[0.5, 0.25, 0.1]       # For Gaussian targets
dw_steps = [0.1, 0.05, 0.025]           # For Double Well

results_dict = {'gaussian': {}, 'gaussian_mixture': {}, 'double_well': {}}
running_times = {'gaussian': {}, 'gaussian_mixture': {}, 'double_well': {}}


# Gaussian sampling
for algo in ['LMC', 'LMCO', 'aHOLLA']:
    results_dict['gaussian'][algo] = {}
    running_times['gaussian'][algo] = {}
    
    for d in d_values:
        results_dict['gaussian'][algo][d] = {}
        running_times['gaussian'][algo][d] = {}
        
        for step_size in gaussian_steps:
            sampler = LangevinSampler(targ='gaussian', algo=algo, step=step_size, d=d)
            theta0 = 2*np.ones(d)

            start_time = time.time()
            samples = draw_samples_parallel(sampler, theta0)
            end_time = time.time()
            
            results_dict['gaussian'][algo][d][step_size] = samples
            runtime = end_time - start_time
            running_times['gaussian'][algo][d][step_size] = runtime
            
            n_iter = len(samples)//n_chains
            print(f"Gaussian| {algo} | d={d} | Stepsize={step_size} | Time={runtime:.2f}s | Iterations={n_iter}")


# Gaussian mixture sampling
for algo in ['LMC', 'LMCO', 'aHOLLA']:
    results_dict['gaussian_mixture'][algo] = {}
    running_times['gaussian_mixture'][algo] = {}
    
    for d in d_values:
        results_dict['gaussian_mixture'][algo][d] = {}
        running_times['gaussian_mixture'][algo][d] = {}
        
        for step_size in gaussian_steps:
            sampler = LangevinSampler(targ='gaussian_mixture',algo=algo,step=step_size,a=0.2 * np.ones(d),d=d)
            theta0 = 2*np.ones(d)
            
            start_time = time.time()
            samples = draw_samples_parallel(sampler, theta0)
            end_time = time.time()
            
            # Store with step_size key
            results_dict['gaussian_mixture'][algo][d][step_size] = samples  
            runtime = end_time - start_time
            running_times['gaussian_mixture'][algo][d][step_size] = runtime
            
            n_iter = len(samples) // n_chains
            print(f"GaussMix| {algo} | d={d} | Stepsize={step_size} | Time={runtime:.2f}s | Iterations={n_iter}")


# Double-well sampling
for algo in ['mTULA', 'HOLA', 'aHOLA']:
    results_dict['double_well'][algo] = {}
    running_times['double_well'][algo] = {}
    
    for d in d_values:
        results_dict['double_well'][algo][d] = {}
        running_times['double_well'][algo][d] = {}
        
        for step_size in dw_steps:
            sampler = LangevinSampler(targ='double_well',algo=algo,step=step_size,d=d)
            theta0 = 2*np.ones(d)
            
            start_time = time.time()
            samples = draw_samples_parallel(sampler, theta0)  
            end_time = time.time()
            
            # Store with step_size key
            results_dict['double_well'][algo][d][step_size] = samples  
            runtime = end_time - start_time
            running_times['double_well'][algo][d][step_size] = runtime
            
            n_iter = len(samples)//n_chains
            print(f"DoubleWell| {algo} | d={d} | Stepsize={step_size} | Time={runtime:.2f}s | Iterations={n_iter}")

In [ ]:
# Displaying the running times in a clear format
print("\nRunning Times Summary:")
for target in running_times:
    print(f"\n{target.capitalize()} Running Times:")
    for algo in running_times[target]:
        print(f"\nAlgorithm: {algo}")
        for d in running_times[target][algo]:
            print(f"  Dimension: {d}")
            for step_size in running_times[target][algo][d]:
                print(f"    Step Size: {step_size} | Time: {running_times[target][algo][d][step_size]:.2f}s")

In [ ]:
def group_samples_fast(samples_df, n_chains):
    vals = samples_df.iloc[:, 0].to_numpy()
    iter_count = len(vals) // n_chains
    return {
        i: vals[np.arange(i, len(vals), iter_count)].tolist()
        for i in range(iter_count)
    }


grouped_results = {}
for target_type in ['gaussian', 'gaussian_mixture', 'double_well']:
    grouped_results[target_type] = {}
    
    # Select algorithms based on target type
    algos = ALGO_LST[:3] if target_type != 'double_well' else ALGO_LST[3:]
    # Select step sizes based on target type
    steps = gaussian_steps if target_type != 'double_well' else dw_steps
    
    for algo in algos:
        grouped_results[target_type][algo] = {}
        
        for d in d_values:
            grouped_results[target_type][algo][d] = {}
            
            for step_size in steps:
                # Get samples dataframe from results dictionary
                df = results_dict[target_type][algo][d][step_size]
                grp = group_samples_fast(df, n_chains)
                grouped_results[target_type][algo][d][step_size] = grp

print(len(grouped_results['gaussian']['LMC']))

In [ ]:
# Function for double well distribution
def calculate_y(x, d):
    def integrand_denominator(r, d):
        return r ** (d / 2 - 1) * np.exp(-(1/4) * r**2 + (r)/2)

    denominator = quad(integrand_denominator, 0, np.inf, args=(d,))[0]

    def integrand_numerator(r):
        return (r ** ((d - 3) / 2) * np.exp(-(1/4) * (r + x**2)**2 + (r + x**2) / 2))

    numerator = quad(integrand_numerator, 0, np.inf)[0]

    y = (gamma(d / 2) / (np.sqrt(np.pi) * gamma((d - 1) / 2))) * numerator / denominator
    return y

# Sampling settings
n_samples = 500
x_grid = np.linspace(-5, 5, 10000)
dx = x_grid[1] - x_grid[0]

# Gaussian samples
gaussian_samples = np.random.randn(n_samples)

# Gaussian mixture samples
mix_centers = np.array([-0.2, 0.2])
component_indices = np.random.choice([0, 1], size=n_samples)
gaussian_mixture_samples = mix_centers[component_indices] + np.random.randn(n_samples)

# Double well samples using inverse method
double_well_samples = {}
for d in d_values:
    pdf_vals = np.array([calculate_y(x, d) for x in x_grid])
    cdf_vals = np.cumsum(pdf_vals) * dx
    cdf_vals /= cdf_vals[-1]  # normalize CDF
    u = np.random.rand(n_samples)
    double_well_samples[d] = np.interp(u, cdf_vals, x_grid)

# Output samples
print("Gaussian samples:", gaussian_samples[:5])
print("Gaussian mixture samples:", gaussian_mixture_samples[:5])
for d in d_values:
    print(f"Double well samples (d={d}):", double_well_samples[d][:5])

In [ ]:
from scipy.stats import wasserstein_distance
gaussian_wasserstein = {}
for algo in ['LMC', 'LMCO', 'aHOLLA']:
    gaussian_wasserstein[algo] = {}
    for d in d_values:
        gaussian_wasserstein[algo][d] = {}
        for step in gaussian_steps:
            gaussian_wasserstein[algo][d][step] = []
            n_iter = int(200 / step * 2)
            for i in range(n_iter):
                algo_samples = grouped_results['gaussian'][algo][d][step][i]
                distance = wasserstein_distance(gaussian_samples, algo_samples)
                gaussian_wasserstein[algo][d][step].append(distance)


gaussian_mixture_wasserstein = {}
for algo in ['LMC', 'LMCO', 'aHOLLA']:
    gaussian_mixture_wasserstein[algo] = {}
    for d in d_values:
        gaussian_mixture_wasserstein[algo][d] = {}
        for step in gaussian_steps:
            gaussian_mixture_wasserstein[algo][d][step] = []
            n_iter = int(200 / step * 2)
            for i in range(n_iter):
                algo_samples = grouped_results['gaussian_mixture'][algo][d][step][i]
                distance = wasserstein_distance(gaussian_mixture_samples, algo_samples)
                gaussian_mixture_wasserstein[algo][d][step].append(distance)


double_well_wasserstein = {}
for algo in ['mTULA', 'HOLA', 'aHOLA']:
    double_well_wasserstein[algo] = {}
    for d in d_values:
        double_well_wasserstein[algo][d] = {}
        true_samples = double_well_samples[d]
        for step in dw_steps:
            double_well_wasserstein[algo][d][step] = []
            n_iter = int(200 / step * 2)
            for i in range(n_iter):
                algo_samples = grouped_results['double_well'][algo][d][step][i]
                distance = wasserstein_distance(true_samples, algo_samples)
                double_well_wasserstein[algo][d][step].append(distance)

In [ ]:
first_below_threshold_gaussian = {
    algo: {
        d: {step: next((i for i, value in enumerate(gaussian_wasserstein[algo][d][step]) if value < 0.05), 'NA') for step in gaussian_steps} 
        for d in d_values
    } 
    for algo in ['LMC', 'LMCO', 'aHOLLA']
}
first_below_threshold_gaussian_mixture = {
    algo: {
        d: {step: next((i for i, value in enumerate(gaussian_mixture_wasserstein[algo][d][step]) if value < 0.05), 'NA') for step in gaussian_steps} 
        for d in d_values
    } 
    for algo in ['LMC', 'LMCO', 'aHOLLA']
}
first_below_threshold_double_well = {
    algo: {
        d: {step: next((i for i, value in enumerate(double_well_wasserstein[algo][d][step]) if value < 0.05), 'NA') for step in dw_steps} 
        for d in d_values
    } 
    for algo in ['mTULA', 'HOLA', 'aHOLA']
}

def print_first_below_threshold_results():
    print("First Below Threshold Gaussian Wasserstein Distances:")
    for algo, data in first_below_threshold_gaussian.items():
        print(f"Algorithm: {algo}")
        for d, steps in data.items():
            print(f"  Dimension: {d}, Steps: {steps}")

    print("\nFirst Below Threshold Gaussian Mixture Wasserstein Distances:")
    for algo, data in first_below_threshold_gaussian_mixture.items():
        print(f"Algorithm: {algo}")
        for d, steps in data.items():
            print(f"  Dimension: {d}, Steps: {steps}")

    print("\nFirst Below Threshold Double Well Wasserstein Distances:")
    for algo, data in first_below_threshold_double_well.items():
        print(f"Algorithm: {algo}")
        for d, steps in data.items():
            print(f"  Dimension: {d}, Steps: {steps}")

# Call the function to print results
print_first_below_threshold_results()

In [ ]:
smallest_gaussian = {
    algo: {
        d: {step: round(min(gaussian_wasserstein[algo][d][step]), 4) for step in gaussian_steps} 
        for d in d_values
    } 
    for algo in ['LMC', 'LMCO', 'aHOLLA']
}
smallest_gaussian_mixture = {
    algo: {
        d: {step: round(min(gaussian_mixture_wasserstein[algo][d][step]), 4) for step in gaussian_steps} 
        for d in d_values
    } 
    for algo in ['LMC', 'LMCO', 'aHOLLA']
}
smallest_double_well = {
    algo: {
        d: {step: round(min(double_well_wasserstein[algo][d][step]), 4) for step in dw_steps} 
        for d in d_values
    } 
    for algo in ['mTULA', 'HOLA', 'aHOLA']
}

def print_results():
    print("Smallest Gaussian Wasserstein Distances:")
    for algo, data in smallest_gaussian.items():
        print(f"Algorithm: {algo}")
        for d, steps in data.items():
            print(f"  Dimension: {d}, Steps: {steps}")

    print("\nSmallest Gaussian Mixture Wasserstein Distances:")
    for algo, data in smallest_gaussian_mixture.items():
        print(f"Algorithm: {algo}")
        for d, steps in data.items():
            print(f"  Dimension: {d}, Steps: {steps}")

    print("\nSmallest Double Well Wasserstein Distances:")
    for algo, data in smallest_double_well.items():
        print(f"Algorithm: {algo}")
        for d, steps in data.items():
            print(f"  Dimension: {d}, Steps: {steps}")

# Call the function to print results
print_results()

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(15, 15))

for i, step in enumerate(gaussian_steps):
    for algo in ALGO_LST[:3]:
        axs[i, 0].plot(gaussian_wasserstein[algo][100][step], label=algo)
    axs[i, 0].set_title(f'Gaussian - Step Size: {step}')
    axs[i, 0].legend()

for i, step in enumerate(gaussian_steps):
    for algo in ALGO_LST[:3]:
        axs[i, 1].plot(gaussian_mixture_wasserstein[algo][100][step], label=algo)
    axs[i, 1].set_title(f'Gaussian Mixture - Step Size: {step}')
    axs[i, 1].legend()

for i, step in enumerate(dw_steps):
    for algo in ALGO_LST[3:]:
        axs[i, 2].plot(double_well_wasserstein[algo][100][step], label=algo)
    axs[i, 2].set_title(f'Double Well - Step Size: {step}')
    axs[i, 2].legend()

plt.tight_layout()
plt.show()